# Assess MaCFP/NIST Parallel Panel Burner

The simulation setup is based on the FDS validation case ["NIST NRC Parallel Panels"/PMMA_60_kW.fds, (commit: b6255e2)](https://github.com/firemodels/fds/blob/b6255e2de7eebe4bfaff8461aad990e7e5eecb98/Validation/NIST_NRC_Parallel_Panels/FDS_Input_Files/PMMA_60_kW.fds), but only contains the empty panels and the burner. It is adopted to incorporate information from the [MaCFP/NIST parallel panel tests](https://github.com/MaCFP/macfp-db/tree/master/Fire_Growth/NIST_Parallel_Panel/Documentation). Different surfaces and materials are define daccording to the experiment description. If material parameters are unknown, the definition of exisitng data (for example: the plywood panel has the same `MATL` as the marinite board; the gravel and sand in the burner are using the definitions from the validation case) are used. 

We are using the [`fdsreader` Python module](https://firedynamics.github.io/LectureFireSimulation/content/tools/03_analysis/02_fdsreader.html) here, to get easy access to the full FDS simulation output, for example the `BNDF`or `SLCF` information.

## Simulation Setups
Overview over a couple of variations on the setup.

- MaCFP_Burner_01: Base setup, FDS default, 2 cm fluid cells 
- MaCFP_Burner_02: `ANGLE_INCREMENT=1`, `TIME_STEP_INCREMENT=1` 
- MaCFP_Burner_03: `PATH_LENGTH=0.8347826086956521`
- MaCFP_Burner_04: `SIMULATION_MODE=LES`
- MaCFP_Burner_05: `SIMULATION_MODE=LES`, `ANGLE_INCREMENT=1`, `TIME_STEP_INCREMENT=1`
- MaCFP_Burner_06: `SIMULATION_MODE=LES`, `PATH_LENGTH=0.8347826086956521`
- MaCFP_Burner_07: 1 cm fluid cells, domain cut to z=1.2 m, `SIMULATION_MODE=VLES`
- MaCFP_Burner_08: 1 cm fluid cells, domain cut to z=1.2 m, `SIMULATION_MODE=LES`


## Data Sets
The primary source of experimental data is the [MaCFP/NIST parallel panel tests](https://github.com/MaCFP/macfp-db/tree/master/Fire_Growth/NIST_Parallel_Panel/Documentation). Main goal is to accurately reproduce the gas burner and its interaction with the empty panels. The resulting simulation setup can later be used to assess the performance of material parameter sets. 

- NIST FDS Validation cases ["NIST/NRC Parallel Panels"](https://github.com/firemodels/exp/tree/master/NIST_NRC_Parallel_Panels) and ["FM Parallel Panel Experiments"](https://github.com/firemodels/fds/tree/master/Validation/FM_Parallel_Panels)
- [MaCFP/NIST parallel panel tests](https://github.com/MaCFP/macfp-db/tree/master/Fire_Growth/NIST_Parallel_Panel/Documentation)



In [ ]:
import os
import pylab
import string
import fdsreader
import matplotlib
import subprocess

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import Processing as proc

from importlib import reload
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage.filters import gaussian_filter


# Reload Python modules for changes that occured during runtime.
# reload(proc)

In [ ]:
print('Package Versions')
print('----------------')
print('Pandas version: {}'.format(pd.__version__))
print('Numpy version: {}'.format(np.__version__))
print('fdsreader version: {}'.format(fdsreader.__version__))
print('Matplotlib version: {}'.format(matplotlib.__version__))

In [ ]:
cwd = os.getcwd()
print(cwd)

In [ ]:
# Get letters for automatic labeling.
letters = list(string.ascii_uppercase)
print(string.ascii_uppercase)


# Path to experiment data.
exp_root = os.path.join("..", "GeneralInformation", "macfp-db")
exp_macfp_dir = os.path.join(exp_root, "Fire_Growth", 
                             "NIST_Parallel_Panel", "Experimental_Data")


# Path to simulation data.
sim_dir = os.path.join("..", "BurnerSims")


# Initialise data collection.
burner_sims = dict()


# Directory used to collect the output produced by this notebook.
output_dir = "AssessBurnerOutput"
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    print("* Output directory created.")
else:
    print("* Output directory already exists.")

In [ ]:
# Basic information about cases.
sim_setup_infos = {
    "MaCFP_Burner_01": {
        "PlotLabel": "Burner_01",
        "PanelID": 2,
        "PanelHeight": [15, 65],
        "PanelWidth": [30],
        "IOR":-1},
    "MaCFP_Burner_02": {
        "PlotLabel": "Burner_02",
        "PanelID": 2,
        "PanelHeight": [15, 65],
        "PanelWidth": [30],
        "IOR":-1},
    "MaCFP_Burner_03": {
        "PlotLabel": "Burner_03",
        "PanelID": 2,
        "PanelHeight": [15, 65],
        "PanelWidth": [30],
        "IOR":-1},
    "MaCFP_Burner_04": {
        "PlotLabel": "Burner_04",
        "PanelID": 2,
        "PanelHeight": [15, 65],
        "PanelWidth": [30],
        "IOR":-1},
    "MaCFP_Burner_05": {
        "PlotLabel": "Burner_05",
        "PanelID": 2,
        "PanelHeight": [15, 65],
        "PanelWidth": [30],
        "IOR":-1},
    "MaCFP_Burner_06": {
        "PlotLabel": "Burner_06",
        "PanelID": 2,
        "PanelHeight": [15, 65],
        "PanelWidth": [30],
        "IOR":-1},
    "MaCFP_Burner_07": {
        "PlotLabel": "Burner_07",
        "PanelID": 2,
        "PanelHeight": [30, 100],
        "PanelWidth": [60],
        "IOR":-1},
    "MaCFP_Burner_08": {
        "PlotLabel": "Burner_08",
        "PanelID": 2,
        "PanelHeight": [30, 100],
        "PanelWidth": [60],
        "IOR":-1},
    "MaCFP_Burner_09": {
        "PlotLabel": "Burner_09",
        "PanelID": 2,
        "PanelHeight": [60, 200],
        "PanelWidth": [120],
        "IOR":-1},
    "MaCFP_Burner_10": {
        "PlotLabel": "Burner_10",
        "PanelID": 2,
        "PanelHeight": [60, 200],
        "PanelWidth": [120],
        "IOR":-1}
}

## Read Experimental Data

In [ ]:
# Get commit hash of MaCFP repo.
# ------------------------------------
# Commit/revision used here:
# MaCFP HEAD, revision (short): b8b0c1b
# MaCFP HEAD, revision  (long): b8b0c1b048f1a1901f2e012463f2a7c168eb486a
# MaCFP description: macfp-1.0-639-gb8b0c1b
# ------------------------------------

# Check where you are at.
# Short form of commit hash.
# (git rev-parse --short HEAD)
revision_s = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], 
                                     cwd=exp_root).strip().decode()

# Long form of commit hash.
# (git rev-parse HEAD)
revision_l = subprocess.check_output(["git", "rev-parse", "HEAD"], 
                                     cwd=exp_root).strip().decode()

# Tag (version) and commit number.
# Needs tags.
description = subprocess.check_output(["git", "describe"], 
                                     cwd=exp_root).strip().decode()


# Show actuals.
print("MaCFP git repo version")
print("------------------------------")
print(f"MaCFP HEAD, revision (short): {revision_s}")
print(f"MaCFP HEAD, revision  (long): {revision_l}")
print(f"MaCFP description: {description}")

In [ ]:
# Get file names.
os.listdir(exp_macfp_dir)

In [ ]:
# Read centre line heat flux to empty panel.
centreline_hf_path = os.path.join(exp_macfp_dir, "Burner_HF_Centerline_multi-layer.csv")
centreline_hf_df = pd.read_csv(centreline_hf_path, header=0, skiprows=[1])


# Check result.
centreline_hf_df.head()

In [ ]:
burner_hrr = {
    "Time": centreline_hf_df.Time,
    "HRR": centreline_hf_df.HRR,
    "u_HRR_min": centreline_hf_df.HRR - centreline_hf_df.u_exp_HRR,
    "u_HRR_max": centreline_hf_df.HRR + centreline_hf_df.u_exp_HRR,
}

burner_hrr["u_HRR_min"][0]

In [ ]:
# Burner heat release rate.
for time_id, time_step in enumerate(burner_hrr["Time"]):
    # Plot error bars.
    plt.plot([time_step, time_step], 
             [burner_hrr["u_HRR_min"][time_id], burner_hrr["u_HRR_max"][time_id]], 
             color='k')

# Plot experiment HRR.
plt.plot(burner_hrr["Time"],
         burner_hrr["HRR"], 
         marker='o', 
         color='k', #alpha=0.6,
         label='Experiment')
    
    
# Plot meta data.
plt.xlabel("Time / s")
plt.ylabel("Energy Release Rate / kW")

plt.xlim(xmin=-5, xmax=90)
plt.ylim(ymin=-10,ymax=110)

plt.rcParams.update({'font.size': 12})

plt.tight_layout()
plt.legend()
plt.grid()


# Save image.
plot_label = f"BurnerHRRExp.png"
plot_path = os.path.join(output_dir, plot_label)
plt.savefig(plot_path, dpi=320, bbox_inches='tight')

# Title after save to just show it here.
plt.title(f"Parallel Panel Burner")
plt.show()

In [ ]:
20/35

In [ ]:
# Read heat flux map to empty panel.
burnersteady_hf_path = os.path.join(exp_macfp_dir, "Burner_steadyHF_Width_multi-layer.csv")
burnersteady_hf_df = pd.read_csv(burnersteady_hf_path, header=0, skiprows=[1])


# Check result.
burnersteady_hf_df.head()

In [ ]:
for line_id, height in enumerate(burnersteady_hf_df["Height"]):
    
    burnersteady_hf_df["HF_y-25"][line_id]
    
    print(height, burnersteady_hf_df["HF_y-25"][line_id])

In [ ]:
hf_gauge = burnersteady_hf_df[["HF_y-25", "HF_y-15", "HF_y0", "HF_y15", "HF_y25"]].to_numpy()


# Set size of the figure.
plt.figure(figsize=(5, 5))

# Create the mesh of the data.
x = np.linspace(-0.25, 0.25, num=5)
y = np.linspace(0.2, 1.00, num=4)
X, Y = np.meshgrid(x, y)
Z = hf_gauge

# sigma = 0.5 # this depends on how noisy your data is, play with it!
# Z = gaussian_filter(Z, sigma)


# Define where the lines are located.
levels = [0, 5, 10, 15, 20, 30, 40, 50, 70]
# Edge length of the data set, i.e. lower part of the panel.
extent = [-0.3,0.3, 0.0,1.0]

# Filled areas.
CS = plt.contourf(X, Y, Z, levels,
                  cmap=plt.cm.viridis)

# Define colour bar.
plt.clim(2.0, 65.0)
plt.colorbar().set_label('Gauge Heat Flux / kW/m²',
                         size=11)


# Contour lines of the gauge heat flux distribution.
contours = plt.contour(X, Y, Z, levels, colors='black')
plt.clabel(contours, levels, inline=True, 
           fmt='%1.0f',  # Set number of digits for contour labels.
           fontsize=9)


# Device locations, experiment.
plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.20,0.20,0.20,0.20,0.20], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.50,0.50,0.50,0.50,0.50], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.75,0.75,0.75,0.75,0.75], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [1.00,1.00,1.00,1.00,1.00], 
            color='k')


# Plot meta data.
plt.xlabel("Y-Direction / m")
plt.ylabel("Z-Direction / m")

plt.xlim(xmin=-0.35, xmax=0.35)
plt.ylim(ymin=-0.02,ymax=1.12)

plt.rcParams.update({'font.size': 12})

plt.tight_layout()
# plt.legend()
# plt.grid()


# Save image.
plot_label = f"BurnerPanelFluxMapExp.png"
plot_path = os.path.join(output_dir, plot_label)
plt.savefig(plot_path, dpi=320, bbox_inches='tight')

# Title after save to just show it here.
plt.title(f"Burner Heat Flux Map, Right Panel (Exp.)")
plt.show()

# 

## Heat Flux to Panel on Vertical Centre Line



In [ ]:
centreline_hf_df.head()

In [ ]:
heights = [0.20,0.50,0.75,1.00]

centreline_fluxes = dict()

for time_id, time_step in enumerate(centreline_hf_df["Time"]):
    hf_centre = centreline_hf_df.iloc[time_id][["HF_z20", "HF_z50", "HF_z75", "HF_z100"]].to_numpy()
    hf_u_centre = centreline_hf_df.iloc[time_id][["u_exp_HF_z20", "u_exp_HF_z50", "u_exp_HF_z75", "u_exp_HF_z100"]].to_numpy()
    print(time_step, hf_centre)
    
    # Create error bars.
    for flux_id, flux in enumerate(hf_centre):
        plt.plot([flux - hf_u_centre[flux_id],flux + hf_u_centre[flux_id]], 
                 [heights[flux_id], heights[flux_id]], color='k')
#         print(hf_u_centre[flux_id], flux)
    
    # Plot data.
    plt.plot(hf_centre, heights, linestyle='--', #marker='o',
             label=f"Exp.: {time_step} s")
    centreline_fluxes[f"{time_step} s"] = [hf_centre, heights]


# Plot meta data.
plt.xlabel("Heat Flux / kW/m²")
plt.ylabel("Centre Line Height / m")

plt.xlim(xmin=-0.5, xmax=70.5)
plt.ylim(ymin=-0.02,ymax=1.12)

# plt.rcParams.update({'font.size': 12})

plt.tight_layout()
plt.legend()
plt.grid()


# Save image.
plot_label = f"BurnerPanelCentreLineHeatFluxExp.png"
plot_path = os.path.join(output_dir, plot_label)
plt.savefig(plot_path, dpi=320, bbox_inches='tight')

# Title after save, to just show it here.
plt.title(f"Vertical Centre Line Heat Flux, Right Panel (Exp.)")
plt.show()

## Read Simulation Response

In [ ]:
# Here the complete FDS output is read, using the fdsreader module.
# The base *_hrr.csv and *_devc.csv are read here individually as well.
# However, this is not really necessary, because all information is 
# available in the simulation object.

# Go over all simulation setups.
for sim_label in os.listdir(sim_dir):
    # Add directory per simulation setup.
    if sim_label not in list(burner_sims):
        # Prevent overwriting existing data.
        burner_sims[sim_label] = dict()
    
    # Show where we are at.
    print(sim_label)
    
    # Create path to FDS simulation output.
    fds_sim_path = os.path.join(sim_dir, sim_label)
    
    # Read *_hrr.csv
    hrr_path = os.path.join(fds_sim_path, 
                            f"{sim_label}_hrr.csv")
    hrr_df = pd.read_csv(hrr_path, header=1)
    # Collect result.
    burner_sims[sim_label]["HRR"] = hrr_df
    
    # Read *_devc.csv
    devc_path = os.path.join(fds_sim_path, 
                            f"{sim_label}_devc.csv")
    devc_df = pd.read_csv(devc_path, header=1)
    # Collect result.
    burner_sims[sim_label]["DEVC"] = devc_df
    
    # Read full FDS sim results.
    fds_data = fdsreader.Simulation(fds_sim_path)
    # Collect result.
    burner_sims[sim_label]["FDS"] = fds_data
    
    

In [ ]:
# Get overview over FDS simulations.
print ("FDS Simulation Overview")
print ("--------------------------")

for sim_label in burner_sims:
    sim_data = burner_sims[sim_label]["FDS"]
    print(f"Basic Info ({sim_label}):")
    print(sim_data)
    obsts = sim_data.obstructions
    
    print("OBST:")
    print(obsts)
    print()

## Check Burner Performance

In [ ]:
# Burner heat release rate.
# Shaded error area.
plt.fill_between(burner_hrr["Time"], 
                 burner_hrr["u_HRR_min"], 
                 burner_hrr["u_HRR_max"],
                 color='k', alpha=0.1,)


# Simulation.
# sim_label = "MaCFP_Burner_01"
for sim_label in burner_sims:
    plot_label = sim_setup_infos[sim_label]["PlotLabel"]
    hrr_data = burner_sims[sim_label]["HRR"]
    plt.plot(hrr_data.Time,
             hrr_data.HRR,
             label=plot_label)
    
    
# Experiment.
for time_id, time_step in enumerate(burner_hrr["Time"]):
    # Plot error bars.
    plt.plot([time_step, time_step], 
             [burner_hrr["u_HRR_min"][time_id], burner_hrr["u_HRR_max"][time_id]], 
             color='k')

plt.plot(centreline_hf_df.Time,
         centreline_hf_df.HRR, 
         marker='o', 
         color='k', #alpha=0.6,
         label='Exp.')
    

# Plot meta data.
plt.xlabel("Time / s")
plt.ylabel("Energy Release Rate / kW")

plt.xlim(xmin=-5, xmax=145)
plt.ylim(ymin=-5,ymax=105)

plt.rcParams.update({'font.size': 12})

plt.tight_layout()
plt.legend()
plt.grid()


# Save image.
plot_label = f"BurnerHRR.png"
plot_path = os.path.join(output_dir, plot_label)
plt.savefig(plot_path, dpi=320, bbox_inches='tight')

# Title after save to just show it here.
plt.title(f"Parallel Panel Burner")
plt.show()

## Assess Heat Flux to Panels

In [ ]:
devc_data = burner_sims[sim_label]["DEVC"]
devc_data.head()

In [ ]:
centreline_hf_df.head()

In [ ]:
# Get overview over DEVC heat flux data.
devc_ids = ["HF_y0_z20", "HF_y0_z50", "HF_y0_z75", "HF_y0_z100"]

for devc_id in devc_ids:

    height_id = f"{devc_id.split('_')[0]}_{devc_id.split('_')[-1]}"

    # Simulation.
    # sim_label = "MaCFP_Burner_01"
    for sim_label in burner_sims:
        plot_label = sim_setup_infos[sim_label]["PlotLabel"]
        # Get DEVC data.
        devc_data = burner_sims[sim_label]["DEVC"]

        plt.plot(devc_data.Time,
                 devc_data[devc_id], 
                 label=plot_label)


    # Experiment.
    plt.plot(centreline_hf_df.Time,
             centreline_hf_df[height_id],
             color="k", marker="o",
             label="Exp.")


    # Plot meta data.
    plt.xlabel("Time / s")
    plt.ylabel("Heat Flux / kW/m²")

    # plt.xlim(xmin=-5, xmax=120)
    # plt.ylim(ymin=-5,ymax=115)

    plt.rcParams.update({'font.size': 12})

    plt.tight_layout()
    plt.legend()
    plt.grid()


    # # Save image.
    # plot_label = f"BurnerHeatFluxDEVC_{devc_id}.png"
    # plot_path = os.path.join(output_dir, plot_label)
    # plt.savefig(plot_path, dpi=320, bbox_inches='tight')

    # Title after save to just show it here.
    plt.title(f"Heat Flux DEVC ({devc_id})")
    plt.show()

In [ ]:
# Get overview over DEVC heat flux data.
devc_ids = ["HF_y0_z20", "HF_y0_z50", "HF_y0_z75", "HF_y0_z100"]
heights = [0.20,0.50,0.75,1.00]
desired_times = [20, 40, 60, 80]  # s




for sim_label in list(burner_sims)[:]:
    # Experiments.
    for time_label in centreline_fluxes:
        hf_data = centreline_fluxes[time_label]
        plt.plot(hf_data[0], hf_data[1], 
                 color='k', alpha=0.6,
                 linestyle='--')
    
    # Simulation.
    for desired_time in desired_times:
        flux_avrgs = list()
        # Define time window to average over.
#         desired_time = 40  # s
        frames = desired_time * 2
        frame_window = 3

        # sim_label = "MaCFP_Burner_01"
        for devc_id in devc_ids:
            # Get DEVC data.
            devc_data = burner_sims[sim_label]["DEVC"]

            # Find time window over the which to average.
            t_min = int(frames - frame_window)
            t_max = int(frames + frame_window + 1)
            # Compute average within the above window.
            flux_avrg = np.average(devc_data[devc_id][t_min:t_max].to_numpy())
            flux_avrgs.append(flux_avrg)
        
#         plt.scatter(devc_data.Time[frames], flux_avrg, color='k')

        plt.plot(flux_avrgs, heights, label=f"Time: {desired_time} s")

    
    # Plot meta data.
    plt.xlabel("Heat Flux / kW/m²")
    plt.ylabel("Centre Line Height / m")

    plt.xlim(xmin=-0.5, xmax=70.5)
    plt.ylim(ymin=-0.02,ymax=1.12)

    # plt.rcParams.update({'font.size': 12})

    plt.tight_layout()
    plt.legend()
    plt.grid()


    # Save image.
    plot_label = f"BurnerPanelCentreLineHeatFlux_{sim_label[:]}.png"
    plot_path = os.path.join(output_dir, plot_label)
    plt.savefig(plot_path, dpi=320, bbox_inches='tight')

    # Title after save to just show it here.
    plot_label = sim_setup_infos[sim_label]["PlotLabel"]
    plt.title(f"Centre Line Heat Flux, Right Panel ({plot_label})")
    plt.show()

In [ ]:
# Stupid, but it works...
devc_locations = {
    "z20": 0, "z50": 1, "z75": 2, "z100": 3, 
    "y-25": 0, "y-15": 1, "y0": 2, "y15": 3, "y25": 4}

In [ ]:
# Create heat flux map from DEVC.
sim_label = "MaCFP_Burner_01"
devc_data = burner_sims[sim_label]["DEVC"]

# Number of points to average, from the end.
n_points = 6

# Initialise data collection.
flux_map = np.zeros((4,5))

for header in list(devc_data):
    # Find heat flux gauges.
    if "HF_" in header:
        y_pos_label = header.split("_")[1]
        z_pos_label = header.split("_")[2]
        
        # Compute average over last n points.
        flux_avrg = np.average(devc_data[header][-n_points:])
        
        # Collect flux average.
        y_pos = devc_locations[y_pos_label]
        z_pos = devc_locations[z_pos_label]
        flux_map[z_pos][y_pos] = flux_avrg

In [ ]:
# Provide data
hf_gauge = flux_map

# Set size of the figure.
plt.figure(figsize=(5, 5))

# Create the mesh of the data.
x = np.linspace(-0.25, 0.25, num=5)
y = np.linspace(0.2, 1.00, num=4)

X, Y = np.meshgrid(x, y)
Z = hf_gauge


# Define where the lines are located.
levels = [0, 5, 10, 15, 20, 30, 40, 50, 70]
# Edge length of the data set, i.e. lower part of the panel.
extent = [-0.25,0.25, 0.2,1.0]

# Filled areas.
CS = plt.contourf(X, Y, Z, levels, extent=extent,
                  cmap=plt.cm.viridis)

# Define colour bar.
plt.clim(2.0, 65.0)
plt.colorbar().set_label('Gauge Heat Flux / kW/m²',
                         size=11)


# Contour lines of the gauge heat flux distribution.
contours = plt.contour(X, Y, Z, levels, extent=extent, colors='black')
plt.clabel(contours, levels, inline=True, 
           fmt='%1.0f',  # Set number of digits for contour labels.
           fontsize=9)


# Device locations, experiment.
plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.20,0.20,0.20,0.20,0.20], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.50,0.50,0.50,0.50,0.50], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.75,0.75,0.75,0.75,0.75], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [1.00,1.00,1.00,1.00,1.00], 
            color='k')

# plt.axis('equal')
# Plot meta data.
plt.xlabel("Y-Direction / m")
plt.ylabel("Z-Direction / m")

plt.xlim(xmin=-0.35, xmax=0.35)
plt.ylim(ymin=-0.05,ymax=1.05)

plt.rcParams.update({'font.size': 12})

plt.tight_layout()
# plt.legend()
# plt.grid()


# Save image.
plot_label = f"BurnerPanelFluxMapSimDEVC_{sim_label}.png"
plot_path = os.path.join(output_dir, plot_label)
plt.savefig(plot_path, dpi=320, bbox_inches='tight')

# Title after save, to just show it here.
plot_label = sim_setup_infos[sim_label]["PlotLabel"]
plt.title(f"Heat Flux Map, Right Panel ({plot_label})")
plt.show()

In [ ]:
# flux_case_infos = {
#     "Case_01": [2, [29, 50], [15, 65]], 
#     "Case_01b": [2, [19,30], [9, 39]], 
#     "Case_01c": [2, [13,20], [6, 26]]}

# flux_cases = dict()

# for flux_case_label in list(flux_case_infos):
#     flux_case_path = os.path.join(burner_setup_dir, flux_case_label)
#     # Read full FDS sim results.
#     flux_case_sim = fdsreader.Simulation(flux_case_path)
#     # Collect result.
#     flux_cases[flux_case_label] = flux_case_sim

In [ ]:
sim_time = 80  # seconds
sim_time_delta = 10  # seconds
quantity = 'GAUGE HEAT FLUX'


# Show info about FDS simulation.
for sim_label in burner_sims:
    print(sim_label)
    # Get FDS simulation results.
    burner_sim = burner_sims[sim_label]["FDS"]

    # Get panel OBST.
    obst = burner_sim.obstructions[sim_setup_infos[sim_label]["PanelID"]]
    
    # Find time step.
    it1 = obst.get_nearest_timestep(sim_time)
    
    # Get array of quantity at specific OBST face.
    ior = sim_setup_infos[sim_label]["IOR"]
    data = obst.get_global_boundary_data_arrays(quantity)[ior]
    print(f'Shape of the data object: {data.shape}')
    
    print(f" Sim. time: {sim_time} s, frame: {it1}")
    obst = burner_sim.obstructions[sim_setup_infos[sim_label]["PanelID"]]
    it = obst.get_nearest_timestep(sim_time + sim_time_delta)
    
    
    print(f" Sim. time: {sim_time} s, frame: {it}")
    print(f" Frame difference: {it-it1}")
    print()

### Boundary Data (BNDF)

In [ ]:
# Create heat flux map, directly from BNDF.

# Info over available frames.
sim_time = 100
# Define the frames over which to average.
frame_min = -20
frame_max = None
# Which BNDF quantity is to be used.
quantity = 'GAUGE HEAT FLUX'


# Which cases to skip.
to_be_skipped = []#["MaCFP_Burner_09", "MaCFP_Burner_10"]

# Simulation.
# sim_label = "MaCFP_Burner_01"
for sim_label in burner_sims:
    print(sim_label)
    print("-------------------")
    if sim_label in to_be_skipped:
        #Skip simulation.
        continue 
        
    # Set size of the figure.
    plt.figure(figsize=(5, 5))
   
    # Get BNDF data set.
    burner_sim = burner_sims[sim_label]["FDS"]  # fdsreader
    obst = burner_sim.obstructions[sim_setup_infos[sim_label]["PanelID"]]
    
    # Get boundary data from correct face of OBST.
    ior = sim_setup_infos[sim_label]["IOR"]
    data = obst.get_global_boundary_data_arrays(quantity)[ior]
    print(f"Number of frames, total: {len(data)}")
    
    # Get time step.
    frame = obst.get_nearest_timestep(sim_time)
    print(f"Sim. time: {sim_time} s, frame: {frame}")
    
    # Define averaging time window.
    print("Averaging window (frames):", frame_min, frame_max)
    
    # Adjust array, because of changes in cell size.
    z_1 = sim_setup_infos[sim_label]["PanelHeight"][0]
    z_2 = sim_setup_infos[sim_label]["PanelHeight"][1]
    print("Panel height (cells): ",z_1, z_2)
    
    # Average over time window.
    bndf_face = np.average(data[frame_min:frame_max], axis=0)
    
    # Cut to lower part of panel.
    bndf_face_map = bndf_face[:,z_1:z_2].T
    print(bndf_face_map.shape)
    print(f'Shape of the data object: {data.shape}')
    
    # Create the mesh of the data: map heat flux to cell locations.
    x = np.linspace(-0.30, 0.30, num=sim_setup_infos[sim_label]["PanelWidth"][0])
    y = np.linspace(0.0, 1.00, num=z_2 - z_1)
    X, Y = np.meshgrid(x, y)
    Z = bndf_face_map

    # Define where the iso-lines are located.
    levels = [0, 5, 10, 15, 20, 30, 40, 50, 70, 100]
    # Edge length of the data set, i.e. lower part of the panel.
    extent = [-0.3,0.3, 0.0,1.0]

    # Map of BNDF data.
    plt.imshow(bndf_face_map, 
               origin='lower', extent=extent,
               cmap=plt.cm.viridis)

    # Define colour bar.
    plt.clim(1.0, 65.0)
    plt.colorbar().set_label('Gauge Heat Flux / kW/m²',
                             size=11)

    # Contour lines of the gauge heat flux distribution.
    contours = plt.contour(X, Y, Z, levels, colors='black')
    plt.clabel(contours, levels, inline=True, 
               fmt='%1.0f',  # Set number of digits for contour labels.
               fontsize=9)

    # Show DEVC locations.
    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.20,0.20,0.20,0.20,0.20], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.50,0.50,0.50,0.50,0.50], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.75,0.75,0.75,0.75,0.75], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [1.00,1.00,1.00,1.00,1.00], 
                color='k')


    # Plot meta data.
    plt.xlabel("Y-Direction / m")
    plt.ylabel("Z-Direction / m")

    plt.xlim(xmin=-0.32, xmax=0.32)
    plt.ylim(ymin=-0.02,ymax=1.12)

    plt.rcParams.update({'font.size': 12})

    plt.tight_layout()
    # plt.legend()
    # plt.grid()


    # Save image.
    plot_label = f"BurnerPanelFluxMapSimBNDF_{sim_label}.png"
    plot_path = os.path.join(output_dir, plot_label)
    plt.savefig(plot_path, dpi=320, bbox_inches='tight')

    # Title after save, to just show it here.
    plot_label = sim_setup_infos[sim_label]["PlotLabel"]
    plt.title(f"Heat Flux Map, Right Panel ({plot_label})")
    plt.show()


In [ ]:
# Create heat flux map from BNDF.
sim_time = 90  # seconds
sim_time_delta = 10  # seconds
avrg_window_half = 10  # average over 20 s, like experiment.
quantity = 'GAUGE HEAT FLUX'

# Which cases to skip.
to_be_skipped = ["MaCFP_Burner_07", "MaCFP_Burner_08",
                 "MaCFP_Burner_09", "MaCFP_Burner_10"]


# Simulation.
# sim_label = "MaCFP_Burner_01"
for sim_label in burner_sims:
    if sim_label in to_be_skipped:
        #Skip simulation.
        continue 
        
    # Set size of the figure.
    plt.figure(figsize=(5, 5))
   
    # Get BNDF data set.
    burner_sim = burner_sims[sim_label]["FDS"]  # fdsreader
    obst = burner_sim.obstructions[sim_setup_infos[sim_label]["PanelID"]]
    
    # Get boundary data from correct face of OBST.
    ior = sim_setup_infos[sim_label]["IOR"]
    data = obst.get_global_boundary_data_arrays(quantity)[ior]
    
    # Get time step.
    frame = obst.get_nearest_timestep(sim_time)
    print(f" Sim. time: {sim_time} s, frame: {frame}")
    
    # Define averaging time window.
    frame_min = obst.get_nearest_timestep(sim_time - avrg_window_half)
    frame_max = obst.get_nearest_timestep(sim_time + avrg_window_half)
    print("Averaging window (frames):", frame_min, frame_max)
    
    # Adjust array, because of changes in cell size.
    z_1 = sim_setup_infos[sim_label]["PanelHeight"][0]
    z_2 = sim_setup_infos[sim_label]["PanelHeight"][1]
    print(z_1, z_2)
    
    # Average over time window.
    bndf_face = np.average(data[frame_min:frame_max],axis=0)
    
    # Cut to lower part of panel
    bndf_face_map = bndf_face[:,z_1:z_2].T
    print(bndf_face_map.shape)
    print(f'Shape of the data object: {data.shape}')
    
    # Create the mesh of the data: map heat flux to cell locations.
    x = np.linspace(-0.30, 0.30, num=sim_setup_infos[sim_label]["PanelWidth"][0])
    y = np.linspace(0.0, 1.00, num=z_2 - z_1)
    X, Y = np.meshgrid(x, y)
    Z = bndf_face_map

    # Define where the iso-lines are located.
    levels = [0, 5, 10, 15, 20, 30, 40, 50, 70, 100]
    # Edge length of the data set, i.e. lower part of the panel.
    extent = [-0.3,0.3, 0.0,1.0]

    # Filled areas.
    CS = plt.contourf(X, Y, Z, levels, extent=extent,
                      cmap=plt.cm.viridis)

    # Define colour bar.
    plt.clim(2.0, 65.0)
    plt.colorbar().set_label('Gauge Heat Flux / kW/m²',
                             size=11)

    # Contour lines of the gauge heat flux distribution.
    contours = plt.contour(X, Y, Z, levels, colors='black')
    plt.clabel(contours, levels, inline=True, 
               fmt='%1.0f',  # Set number of digits for contour labels.
               fontsize=9)

    # Show DEVC locations.
    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.20,0.20,0.20,0.20,0.20], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.50,0.50,0.50,0.50,0.50], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.75,0.75,0.75,0.75,0.75], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [1.00,1.00,1.00,1.00,1.00], 
                color='k')


    # Plot meta data.
    plt.xlabel("Y-Direction / m")
    plt.ylabel("Z-Direction / m")

    plt.xlim(xmin=-0.32, xmax=0.32)
    plt.ylim(ymin=-0.02,ymax=1.12)

    plt.rcParams.update({'font.size': 12})

    plt.tight_layout()
    # plt.legend()
    # plt.grid()


    # Save image.
    plot_label = f"BurnerPanelFluxMapSimAvrgBNDF_{sim_label}.png"
    plot_path = os.path.join(output_dir, plot_label)
    plt.savefig(plot_path, dpi=320, bbox_inches='tight')

    # Title after save, to just show it here.
    plot_label = sim_setup_infos[sim_label]["PlotLabel"]
    plt.title(f"Heat Flux Map, Right Panel ({plot_label})")
    plt.show()


In [ ]:
burner_sims["MaCFP_Burner_01"]["DEVC"].Time[200]

In [ ]:
# Create heat flux map from DEVC.

# Which cases to skip.
to_be_skipped = ["MaCFP_Burner_08", "MaCFP_Burner_09", "MaCFP_Burner_10"]

# Number of points to average, from the end.
n_points = 4

# Simulation.
# sim_label = "MaCFP_Burner_01"
for sim_label in burner_sims:
    if sim_label in to_be_skipped:
        #Skip simulation.
        continue 
        
    # Set size of the figure.
    plt.figure(figsize=(5, 5))
    
    # Get DEVC data.
    devc_data = burner_sims[sim_label]["DEVC"]
    
    # Initialise data collection.
    flux_map = np.zeros((4,5))
    
    for header in list(devc_data):
        # Find heat flux gauges.
        if "HF_" in header:
            y_pos_label = header.split("_")[1]
            z_pos_label = header.split("_")[2]
        
            # Compute average over last n points.
            flux_avrg = np.average(devc_data[header][160:200])
            
            # Collect flux average.
            y_pos = devc_locations[y_pos_label]
            z_pos = devc_locations[z_pos_label]
            flux_map[z_pos][y_pos] = flux_avrg
    
    # Provide data
    hf_gauge = flux_map

    # Create the mesh of the data.
    # x = np.linspace(-0.25, 0.25, num=5)
    # y = np.linspace(0.2, 1.00, num=4)
    x = np.array([-0.25, -0.15, 0.00, 0.15, 0.25])
    y = np.array([0.20, 0.50, 0.75, 1.00])

    X, Y = np.meshgrid(x, y)
    Z = hf_gauge

    # Define where the lines are located.
    levels = [0, 5, 10, 15, 20, 30, 40, 50, 70]

    # Filled areas.
    CS = plt.contourf(X, Y, Z, levels,
                      cmap=plt.cm.viridis)

    # Define colour bar.
    plt.clim(2.0, 65.0)
    plt.colorbar().set_label('Gauge Heat Flux / kW/m²',
                             size=11)


    # Contour lines of the gauge heat flux distribution.
    contours = plt.contour(X, Y, Z, levels, colors='black')
    plt.clabel(contours, levels, inline=True, 
               fmt='%1.0f',  # Set number of digits for contour labels.
               fontsize=9)

    # Show DEVC locations.
    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.20,0.20,0.20,0.20,0.20], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.50,0.50,0.50,0.50,0.50], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.75,0.75,0.75,0.75,0.75], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [1.00,1.00,1.00,1.00,1.00], 
                color='k')


    # Plot meta data.
    plt.xlabel("Y-Direction / m")
    plt.ylabel("Z-Direction / m")

    plt.xlim(xmin=-0.32, xmax=0.32)
    plt.ylim(ymin=-0.02,ymax=1.12)

    plt.rcParams.update({'font.size': 12})

    plt.tight_layout()
    # plt.legend()
    # plt.grid()


    # Save image.
    plot_label = f"BurnerPanelFluxMapSimDEVC_{sim_label}.png"
    plot_path = os.path.join(output_dir, plot_label)
    plt.savefig(plot_path, dpi=320, bbox_inches='tight')

    # Title after save, to just show it here.
    plot_label = sim_setup_infos[sim_label]["PlotLabel"]
    plt.title(f"Heat Flux Map, Right Panel ({plot_label})")
    plt.show()


In [ ]:
# Create heat flux map from DEVC.

# Number of points to average, from the end.
n_points = 6

# Initialise data collection.
flux_map = np.zeros((4,5))

for header in list(devc_data):
    # Find heat flux gauges.
    if "HF_" in header:
        y_pos_label = header.split("_")[1]
        z_pos_label = header.split("_")[2]
        
        # Compute average over last n points.
        flux_avrg = np.average(devc_data[header][-n_points:])
        
        # Collect flux average.
        y_pos = devc_locations[y_pos_label]
        z_pos = devc_locations[z_pos_label]
        flux_map[z_pos][y_pos] = flux_avrg

In [ ]:
# Create colormap to identify differences in heat map.

# https://matplotlib.org/2.0.2/examples/pylab_examples/custom_cmap.html
cdict2 = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 0.0, 1.0),
                   (1.0, 0.1, 1.0)),
         'green': ((0.0, 0.0, 0.0),
                   (1.0, 0.0, 0.0)),
         'blue':  ((0.0, 0.0, 0.1),
                   (0.5, 1.0, 0.0),
                   (1.0, 0.0, 0.0))
        }
blue_red2 = LinearSegmentedColormap('BlueRed2', cdict2)
plt.register_cmap(cmap=blue_red2)

In [ ]:
# Create heat flux map differences from DEVC (exp - sim).

# Experiment.
hf_gauge = burnersteady_hf_df[["HF_y-25", "HF_y-15", "HF_y0", "HF_y15", "HF_y25"]].to_numpy()
# Simulation
hf_sim = flux_map


# Create the mesh of the data.
# x = np.linspace(-0.25, 0.25, num=5)
# y = np.linspace(0.2, 1.00, num=4)
x = np.array([-0.25, -0.15, 0.00, 0.15, 0.25])
y = np.array([0.20, 0.50, 0.75, 1.00])

X, Y = np.meshgrid(x, y)
# Experiment - simulation
Z = hf_gauge - hf_sim


# Define where the lines are located.
levels = [-40,-30,-20,-10,0,10,20,30,40]

# Filled areas, differences.
cmap = plt.get_cmap('BlueRed2')
CS = plt.contourf(X, Y, Z, levels,
                  cmap=cmap)

# Define colour bar.
plt.clim(levels[0], levels[-1])
plt.colorbar().set_label('Gauge Heat Flux / kW/m²',
                         size=11)


# Contour lines of the gauge heat flux distribution.
contours = plt.contour(X, Y, Z, levels, colors='black')
plt.clabel(contours, levels, inline=True, 
           fmt='%1.0f',  # Set number of digits for contour labels.
           fontsize=9)


plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.20,0.20,0.20,0.20,0.20], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.50,0.50,0.50,0.50,0.50], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.75,0.75,0.75,0.75,0.75], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [1.00,1.00,1.00,1.00,1.00], 
            color='k')


# Plot meta data.
plt.xlabel("Y-Direction / m")
plt.ylabel("Z-Direction / m")

plt.xlim(xmin=-0.32, xmax=0.32)
plt.ylim(ymin=-0.02,ymax=1.12)

plt.rcParams.update({'font.size': 12})

plt.tight_layout()
# plt.legend()
# plt.grid()


# Save image.
plot_label = f"BurnerPanelFluxMapDiff.png"
plot_path = os.path.join(output_dir, plot_label)
plt.savefig(plot_path, dpi=320, bbox_inches='tight')

# Title after save to just show it here.
plt.title(f"Burner Heat Flux Map, Right Panel (Difference)")
plt.show()

In [ ]:
# Create heat flux map differences from DEVC (exp - sim).
# Red: experiment is larger
# Blue: simulation is larger

# Which cases to skip.
to_be_skipped = ["MaCFP_Burner_09", "MaCFP_Burner_10"]

# Number of points to average, from the end.
n_points = 6

# Experiment data.
hf_exp = burnersteady_hf_df[["HF_y-25", "HF_y-15", "HF_y0", "HF_y15", "HF_y25"]].to_numpy()



# Simulation.
# sim_label = "MaCFP_Burner_01"
for sim_label in burner_sims:
    if sim_label in to_be_skipped:
        #Skip simulation.
        continue 
        
    # Set size of the figure.
    plt.figure(figsize=(5, 5))
    
    # Get DEVC data.
    devc_data = burner_sims[sim_label]["DEVC"]
    
    # Initialise data collection.
    flux_map = np.zeros((4,5))
    
    for header in list(devc_data):
        # Find heat flux gauges.
        if "HF_" in header:
            y_pos_label = header.split("_")[1]
            z_pos_label = header.split("_")[2]
        
            # Compute average over last n points.
            flux_avrg = np.average(devc_data[header][-n_points:])
            
            # Collect flux average.
            y_pos = devc_locations[y_pos_label]
            z_pos = devc_locations[z_pos_label]
            flux_map[z_pos][y_pos] = flux_avrg
    
    # Provide data
    hf_sim = flux_map

    # Create the mesh of the data.
    # x = np.linspace(-0.25, 0.25, num=5)
    # y = np.linspace(0.2, 1.00, num=4)
    x = np.array([-0.25, -0.15, 0.00, 0.15, 0.25])
    y = np.array([0.20, 0.50, 0.75, 1.00])
    
    X, Y = np.meshgrid(x, y)
    # Experiment - simulation
    Z = hf_exp - hf_sim
    

    # Define where the lines are located.
    levels = [-50,-40,-30,-20,-10,-4,-2,0,2,4,10,20,30,40,50]

    # Filled areas, differences.
    cmap = plt.get_cmap('BlueRed2')
    CS = plt.contourf(X, Y, Z, levels,
                      cmap=cmap)

    # Define colour bar.
    plt.clim(levels[0], levels[-1])
    plt.colorbar().set_label('Gauge Heat Flux / kW/m²',
                             size=11)


    # Contour lines of the gauge heat flux distribution.
    contours = plt.contour(X, Y, Z, levels, colors='black')
    plt.clabel(contours, levels, inline=True, 
               fmt='%1.0f',  # Set number of digits for contour labels.
               fontsize=9)

    # Show DEVC locations.
    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.20,0.20,0.20,0.20,0.20], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.50,0.50,0.50,0.50,0.50], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [0.75,0.75,0.75,0.75,0.75], 
                color='k')

    plt.scatter([-0.25,-0.15,0,0.15,0.25],
                [1.00,1.00,1.00,1.00,1.00], 
                color='k')


    # Plot meta data.
    plt.xlabel("Y-Direction / m")
    plt.ylabel("Z-Direction / m")

    plt.xlim(xmin=-0.32, xmax=0.32)
    plt.ylim(ymin=-0.02,ymax=1.12)

    plt.rcParams.update({'font.size': 12})

    plt.tight_layout()
    # plt.legend()
    # plt.grid()


    # Save image.
    plot_label = f"BurnerPanelFluxMapDiff_{sim_label}.png"
    plot_path = os.path.join(output_dir, plot_label)
    plt.savefig(plot_path, dpi=320, bbox_inches='tight')

    # Title after save, to just show it here.
    plt.title(f"Heat Flux Map, Right Panel ({sim_label})")
    plt.show()

    
    
    

In [ ]:
stop_here

# Work with fdsreader

In [ ]:
sim_label = "MaCFP_Burner_01"
fds_sim = burner_sims[sim_label]["FDS"]

fds_sim

In [ ]:
fds_sim.obstructions

In [ ]:
for obst in fds_sim.obstructions:
    print(f'ID: {obst.id}; has data: {obst.has_boundary_data}')
    if obst.has_boundary_data:
        print(f'  orientations: {obst.orientations}')
        print(f'  quantities: {obst.quantities}')
        print(f'  number of involved meshes: {len(obst.meshes)}')

In [ ]:
obst = fds_sim.obstructions.get_by_id(2)
print(obst)

In [ ]:
obst = fds_sim.obstructions[2]
it = obst.get_nearest_timestep(80)
print(it)
data = obst.get_global_boundary_data_arrays('GAUGE HEAT FLUX')[-1]
print(f'shape of the data object: {data.shape}')

In [ ]:
extent = [-0.3,0.3, 0.0,1.0]

plt.imshow(data[it][:,15:65].T, 
           origin='lower', extent=extent)
plt.colorbar()


# Device locations, experiment.
plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.20,0.20,0.20,0.20,0.20], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.50,0.50,0.50,0.50,0.50], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.75,0.75,0.75,0.75,0.75], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [1.00,1.00,1.00,1.00,1.00], 
            color='k')


# Plot meta data.
plt.xlabel("Y-Direction / m")
plt.ylabel("Z-Direction / m")

plt.xlim(xmin=-0.35, xmax=0.35)
plt.ylim(ymin=-0.05,ymax=1.05)

plt.rcParams.update({'font.size': 12})

plt.tight_layout()
# plt.legend()
# plt.grid()


# Save image.
plot_label = f"BurnerPanelFluxMapSimBNDF_{sim_label}.png"
plot_path = os.path.join(output_dir, plot_label)
plt.savefig(plot_path, dpi=320, bbox_inches='tight')

# Title after save, to just show it here.
plt.title(f"Heat Flux Map, Right Panel ({sim_label})")
plt.show()



In [ ]:
data[40:42][:,15:65].shape

In [ ]:
np.average(data[38:42],axis=0)[:,15:65].T.shape

In [ ]:
data[it][:,15:65].T.shape

In [ ]:
sim_time = 80  # seconds
obst = fds_sim.obstructions[2]
it = obst.get_nearest_timestep(sim_time)
print(f" Sim. time: {sim_time} s, frame: {it}")

In [ ]:
# Provide data
frame = 42
avrg_window = 3
frame_min = frame - avrg_window
frame_max = frame + avrg_window
print(frame_min, frame_max)
hf_gauge = np.average(data[frame_min:frame_max],axis=0)[:,15:65].T

# Create the mesh of the data.
x = np.linspace(-0.30, 0.30, num=30)
y = np.linspace(0.0, 1.00, num=50)
X, Y = np.meshgrid(x, y)
Z = hf_gauge

# Define where the lines are located.
levels = [0, 5, 10, 15, 20, 30, 40, 50, 70]

extent = [-0.3,0.3, 0.0,1.0]


plt.imshow(hf_gauge, 
           origin='lower', extent=extent)

# Define colour bar.
plt.clim(2.0, 65.0)
plt.colorbar().set_label('Gauge Heat Flux / kW/m²',
                         size=11)


# Contour lines of the gauge heat flux distribution.
contours = plt.contour(X, Y, Z, levels, colors='black')
plt.clabel(contours, levels, inline=True, 
           fmt='%1.0f',  # Set number of digits for contour labels.
           fontsize=9)




# Device locations, experiment.
plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.20,0.20,0.20,0.20,0.20], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.50,0.50,0.50,0.50,0.50], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [0.75,0.75,0.75,0.75,0.75], 
            color='k')

plt.scatter([-0.25,-0.15,0,0.15,0.25],
            [1.00,1.00,1.00,1.00,1.00], 
            color='k')


# Plot meta data.
plt.xlabel("Y-Direction / m")
plt.ylabel("Z-Direction / m")

plt.xlim(xmin=-0.35, xmax=0.35)
plt.ylim(ymin=-0.05,ymax=1.05)

plt.rcParams.update({'font.size': 12})

plt.tight_layout()
# plt.legend()
# plt.grid()


# Save image.
plot_label = f"BurnerPanelFluxMapSimBNDF_{sim_label}.png"
plot_path = os.path.join(output_dir, plot_label)
plt.savefig(plot_path, dpi=320, bbox_inches='tight')

# Title after save, to just show it here.
plt.title(f"Heat Flux Map, Right Panel ({sim_label})")
plt.show()

